In [72]:
import pandas as pd
import numpy as np

First we'll save the 'counts' dataframe as its own csv. This python code was taken from GO_term_analysis.py

In [73]:
## Save counts as its own csv

counts = pd.read_csv('htseq_counts_stranded_reverse.txt', sep='\t')
counts = counts.drop(['Gene_2', 'Gene_3', 'Gene_4'], axis=1)
counts = counts.loc[~((counts['AF16_Control'] == 0) & (counts['AF16_Treatment'] == 0))]
counts = counts.loc[~((counts['HK104_Control'] == 0) & (counts['HK104_Treatment'] == 0))]
counts = counts.replace(0,1)
counts['log2foldchange_control'] = counts['AF16_Control']/counts['HK104_Control']
counts['log2foldchange_control'] = np.log2(counts.log2foldchange_control)
counts['log2foldchange_treatment'] = counts['AF16_Treatment']/counts['HK104_Treatment']
counts['log2foldchange_treatment'] = np.log2(counts.log2foldchange_treatment)
counts.to_csv('experimental_data.csv')

Now we can load all the relevant data into pandas dataframes. In the below cell we'll load what was the 'counts' dataframe into experiment, and clean it up. We'll use only the gene name, control, and treatment values, and drop all the other columns. I divided control by treatment, took the absolute value, and stuck that in the test_value column.

In [74]:
experiment = pd.read_csv('experimental_data.csv')

experiment = experiment[['Gene', 'log2foldchange_control', 'log2foldchange_treatment']].copy()
experiment = experiment.rename({'log2foldchange_control': 'control', 'log2foldchange_treatment':'treatment'}, axis=1)
experiment['test_value'] = np.abs(experiment['control'] / experiment['treatment'])
experiment

,Gene,control,treatment,test_value
0,WBGene00000307,-0.081743,-0.505955,0.161562
1,WBGene00000312,-0.289507,1.415037,0.204593
2,WBGene00000324,-0.192645,1.678072,0.114801
3,WBGene00000328,0.129283,0.389152,0.332217
4,WBGene00000329,0.129080,-0.156869,0.822853
...,...,...,...,...
14138,WBGene00304233,-2.321928,-3.584963,0.647685
14139,WBGene00304234,-0.304855,0.917538,0.332253
14140,WBGene00304757,0.241008,0.961526,0.250652
14141,WBGene00304763,2.321928,0.000000,inf


Now we'll load 'GO_terms_no_dups.txt' into the df go_genes. I parsed it so that in the 'genes' column I have a list of all the genes corresponding to that particular go squence.

In [75]:
go_genes = pd.read_csv('briggsae_GO_terms_no_dups.txt',sep='\t')
go_genes = go_genes.rename({go_genes.columns[0]:'go_sequence',
                            go_genes.columns[1]:'genes'}, axis=1)
go_genes

,go_sequence,genes
0,GO:0000003,"WBGene00023826,WBGene00024356,WBGene00024943,W..."
1,GO:0000012,WBGene00041049
2,GO:0000019,WBGene00035859
3,GO:0000022,WBGene00026971
4,GO:0000027,"WBGene00023936,WBGene00024087,WBGene00024572,W..."
...,...,...
2877,GO:2001255,WBGene00033437
2878,GO:2001256,WBGene00025899
2879,GO:2001258,WBGene00036655
2880,GO:2001294,WBGene00037538


Now we're going to remove all genes in experiment that don't have a go_sequence.

In [76]:
def find_my_GO(string):
    t = go_genes.loc[go_genes['genes'].str.contains(string)]
    return t['go_sequence'].str.cat(sep=',')

experiment['go_sequences'] = np.vectorize(find_my_GO)(experiment['Gene'])


In [77]:
experiment = experiment.loc[experiment['go_sequences'] != '']
experiment = experiment.loc[experiment['test_value'] == experiment['test_value']]
experiment

,Gene,control,treatment,test_value,go_sequences
1,WBGene00000312,-0.289507,1.415037,0.204593,"GO:0008150,GO:0015671"
2,WBGene00000324,-0.192645,1.678072,0.114801,"GO:0006811,GO:0006813,GO:0051260,GO:0055085"
3,WBGene00000328,0.129283,0.389152,0.332217,GO:0006470
4,WBGene00000329,0.129080,-0.156869,0.822853,"GO:0007275,GO:0007283,GO:0007548,GO:0010628,GO..."
5,WBGene00000332,0.217002,0.072512,2.992621,"GO:0006006,GO:0006096,GO:0055114"
...,...,...,...,...,...
14069,WBGene00303306,4.990955,2.392317,2.086243,GO:0006468
14088,WBGene00303376,1.000000,1.514573,0.660252,"GO:0007155,GO:0007156"
14111,WBGene00303438,-0.082305,-0.020122,4.090396,"GO:0006465,GO:0006508,GO:0006627"
14122,WBGene00304200,0.145648,0.093109,1.564263,GO:0006468


In [79]:
def count_genes(go_seq):
    t = experiment.loc[experiment['go_sequences'].str.contains(go_seq)]
    
    return t.shape[0]
    
go_genes['count'] = np.vectorize(count_genes)(go_genes['go_sequence'])
go_genes = go_genes.iloc[:2881, :].copy()

go_genes = go_genes.loc[go_genes['count'] != 0]

Now we add the p values.

In [80]:
def test_avg(go_seq):
    t = experiment.loc[experiment['go_sequences'].str.contains(go_seq)]
    return np.mean(t['test_value'])

go_genes['test_avg'] = np.vectorize(test_avg)(go_genes['go_sequence'])
go_genes

t = experiment.loc[experiment['go_sequences'].str.contains('GO:0000003')]

In [81]:
def p_value(go_seq):
    t = experiment.loc[~experiment['go_sequences'].str.contains(go_seq)]
    sample_number = go_genes.loc[go_genes['go_sequence']==go_seq]['count'].values[0]
    test_mean = go_genes.loc[go_genes['go_sequence']==go_seq]['test_avg'].values[0]
    means = []
    for i in range(1000):
        samples = np.random.choice(a=np.array(t['test_value']), size=sample_number)
        sample_mean = np.mean(samples)
        means += [sample_mean]
    means = np.array(means)
    return np.sum(test_mean >= means) / 1000



go_genes['p_value'] = np.vectorize(p_value)(go_genes['go_sequence'])


In [82]:
go_genes[['go_sequence', 'p_value']].to_csv('pvalues.csv')